# Execução local
Este notebook é uma adaptação do original usado no Google Colab, preparada para rodar no seu ambiente local e permitir a atualização do BigQuery.


# ETL para carregar o PIB, População e o Centróide dos municípios


In [ ]:
# Configuração do ambiente local
import importlib
import subprocess
import sys
from pathlib import Path

REQUIRED_PACKAGES = {
    "pandas": "pandas",
    "dbf": "dbf",
    "pandas_gbq": "pandas-gbq",
    "google.oauth2": "google-auth",
    "google.cloud": "google-cloud-bigquery",
    "openpyxl": "openpyxl",
}

for module_name, package_name in REQUIRED_PACKAGES.items():
    try:
        importlib.import_module(module_name)
    except ImportError:
        print(f"Instalando pacote '{package_name}'...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

import pandas as pd
from pandas.io import gbq
print("Ambiente local configurado.")


## Pré-processamento dos centroides dos municipios


### Solução 1 - Processamento do arquivo de banco de dados (dbf)




In [ ]:
from pathlib import Path
import dbf

DATA_DIR = Path.cwd()
print(f"Diretório de trabalho: {DATA_DIR}")
print(f"Biblioteca 'dbf' carregada: versão {getattr(dbf, '__version__', 'desconhecida')}")


In [ ]:
from pathlib import Path
from urllib.request import urlretrieve

DBF_URL = "https://github.com/renatocol/Latitude_Longitude_Brasil/raw/master/BR_Localidades_2010.dbf"
DBF_PATH = Path("BR_Localidades_2010.dbf")

if not DBF_PATH.exists():
    print(f"Baixando {DBF_PATH.name}...")
    urlretrieve(DBF_URL, DBF_PATH)
    print(f"Download concluído: {DBF_PATH.resolve()}")
else:
    size = DBF_PATH.stat().st_size
    print(f"Arquivo {DBF_PATH.name} já existe ({size:,} bytes).")


In [ ]:
from pathlib import Path

print("Arquivos no diretório atual:")
for entry in sorted(Path.cwd().iterdir()):
    if entry.is_file():
        print(f"- {entry.name} ({entry.stat().st_size:,} bytes)")
    else:
        print(f"- {entry.name}/")


In [ ]:
import dbf
import pandas as pd
from pathlib import Path

DBF_PATH = Path("BR_Localidades_2010.dbf")
if not DBF_PATH.exists():
    raise FileNotFoundError("Arquivo BR_Localidades_2010.dbf não encontrado. Execute a célula de download antes.")

table = dbf.Table(str(DBF_PATH))
table.open(dbf.READ_ONLY)
try:
    df = pd.DataFrame(table)
finally:
    table.close()

print(f"Registros carregados: {df.shape[0]} linhas x {df.shape[1]} colunas")


In [ ]:
df.head()


In [ ]:
# Equivalente SQL: select col9 as cod_ibge, col16 as categoria, col18 as long, col19 as lat from df
df_geo = df[[9,16,18,19]].rename(columns={9:"cod_ibge", 16:"categoria", 18:"long", 19:"lat"})
df_geo['cod_ibge'] = df_geo['cod_ibge'].str.strip()
df_geo['categoria'] = df_geo['categoria'].str.strip()
df_geo = df_geo[df_geo['categoria']=='CIDADE']
df_geo.info()


In [ ]:
df_geo['lat_long'] = df_geo[['lat','long']].apply(lambda x: f"{str(x['lat']).replace(',','.')},{str(x['long']).replace(',','.')}", axis=1)
df_geo.head()


In [ ]:
cat = 'CIDADE      '
cat.strip()


### Solução 2 - Converter o arquivo dbf para xlsx usando o excel. Processar o arquivo xlsx com o pandas
#### Vantagem: mais rápido do que o pandas processar o arquivo dbf.
#### URL do arquivo exportado para xlsx https://github.com/alexlopespereira/mba_enap/raw/refs/heads/main/data/originais/centroide_municipios/BR_Localidades_2010_v1.xlsx


In [ ]:
# Use a biblioteca pandas.
import pandas as pd
# Leia o arquivo Excel disponível em: # https://github.com/alexlopespereira/mba_enap/raw/refs/heads/main/data/originais/centroide_municipios/BR_Localidades_2010_v1.xlsx
# Garanta que a coluna "CD_GEOCODM,C,20" seja importada como texto (str).
df_xlsx = pd.read_excel('https://github.com/alexlopespereira/mba_enap/raw/refs/heads/main/data/originais/centroide_municipios/BR_Localidades_2010_v1.xlsx', dtype={'CD_GEOCODM,C,20':str})
# Mostre as 5 primeiras linhas do dataframe.
df_xlsx.head()


In [ ]:
df_xlsx.columns


In [ ]:
# Crie um novo dataframe chamado dfxlsx_geo contendo apenas as colunas:
# "CD_GEOCODM,C,20" → renomeada para "cod_ibge"
# "NM_CATEGOR,C,50" → renomeada para "categoria"
# "LONG,N,24,6" → renomeada para "long"
# "LAT,N,24,6" → renomeada para "lat"

dfxlsx_geo = df_xlsx[['CD_GEOCODM,C,20','NM_CATEGOR,C,50','LONG,N,24,6','LAT,N,24,6']].rename(columns={'CD_GEOCODM,C,20':"cod_ibge", 'NM_CATEGOR,C,50':"categoria", 'LONG,N,24,6':"long", 'LAT,N,24,6':"lat"})
dfxlsx_geo.head()


In [ ]:
dfxlsx_geo['categoria'] = dfxlsx_geo['categoria'].str.strip()
dfxlsx_geo = dfxlsx_geo[dfxlsx_geo['categoria']=='CIDADE']
dfxlsx_geo.loc[:, 'lat_long'] = dfxlsx_geo['lat'].astype(str) + ',' + dfxlsx_geo['long'].astype(str)
dfxlsx_geo.head()


## Join com a tabela de PIB per capita


### Configurar credenciais locais do BigQuery
Defina a variável de ambiente `GOOGLE_APPLICATION_CREDENTIALS` com o caminho do arquivo JSON da conta de serviço ou coloque o arquivo `service-account.json` neste diretório antes de executar as consultas.


In [ ]:
import os
from pathlib import Path
from google.oauth2 import service_account

credentials = None

service_account_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if service_account_path:
    candidate = Path(service_account_path)
    if candidate.exists():
        credentials = service_account.Credentials.from_service_account_file(candidate)
        print(f"Credenciais carregadas de {candidate}")
    else:
        print(f"⚠ Arquivo informado em GOOGLE_APPLICATION_CREDENTIALS não encontrado: {candidate}")

if credentials is None:
    local_candidate = Path("service-account.json")
    if local_candidate.exists():
        credentials = service_account.Credentials.from_service_account_file(local_candidate)
        print(f"Credenciais carregadas de {local_candidate.resolve()}")
    else:
        print("⚠ Nenhum arquivo de credencial localizado. Configure antes de acessar o BigQuery.")


In [ ]:
import os
from pandas.io import gbq

if 'credentials' not in globals() or credentials is None:
    raise RuntimeError("Configure as credenciais do BigQuery antes de executar esta célula.")

project_id = os.getenv("GOOGLE_CLOUD_PROJECT", "365846072239")
print(f"Usando projeto GCP: {project_id}")

query = '''
                        SELECT
                          d.id_municipio,
                          d.nome AS nome_municipio,
                          d.sigla_uf,
                          pop.ano,
                          PARSE_DATE('%Y', CAST(pop.ano AS STRING)) AS data_ano,
                          pop.populacao,
                          pib.pib,
                          ROUND(SAFE_DIVIDE(pib.pib, pop.populacao), 2) AS pib_per_capita
                        FROM `basedosdados.br_bd_diretorios_brasil.municipio` AS d

                        -- junta com população (se não tiver população, fica NULL)
                        LEFT JOIN `basedosdados.br_ibge_populacao.municipio` AS pop
                          ON d.id_municipio = pop.id_municipio
                          AND pop.ano BETWEEN 2002 AND 2018

                        -- junta com pib (se não tiver pib, fica NULL)
                        LEFT JOIN `basedosdados.br_ibge_pib.municipio` AS pib
                          ON d.id_municipio = pib.id_municipio
                        AND pop.ano = pib.ano
                        AND pop.ano BETWEEN 2002 AND 2018
                        --LIMIT 100
                        ;
                        '''

df_pibpercapita = gbq.read_gbq(query, project_id=project_id, credentials=credentials)
df_pibpercapita.head()


In [ ]:
df_pibpercapita['id_municipio'].unique().size


In [ ]:
dfxlsx_geo['cod_ibge'].unique().size


In [ ]:
#Algebra relacional: Join. Ou seja, junção de tabelas.
df_merge = df_pibpercapita.merge(dfxlsx_geo[['cod_ibge','lat_long']], how='left', left_on='id_municipio', right_on='cod_ibge')
df_merge.head()

# renomear pibpercapita para pib_per_capita
df_merge.rename(columns={'pibpercapita': 'pib_per_capita'}, inplace=True)
df_merge.head()

# formate a PARSE_DATE('%Y', CAST(pop.ano AS STRING)) AS data_ano,


In [ ]:
del df_merge['cod_ibge']
df_merge.head()


In [ ]:
print(f"Projeto ativo para BigQuery: {project_id}")


In [ ]:
if 'credentials' not in globals() or credentials is None:
    raise RuntimeError("Configure as credenciais do BigQuery antes de executar esta c?lula.")

destination_table = "enapdatasets.pibpercapita"
df_merge.to_gbq(
    destination_table,
    project_id=project_id,
    chunksize=40000,
    if_exists='replace',
    credentials=credentials,
)
print(f"Tabela atualizada no BigQuery: {destination_table}")
